Final Preprocessing Notebook

In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [2]:
def preprocess_and_group_columns(df, columns_to_remove=None, prefixes_to_remove=None):
    """
    This function will take in columns and prefixes that the user feeds in to remove from the dataframe.
    It will then replace negative values (except -6 and -8) with NaN. 
    Finally it will print out how many columns were in each value pattern. 
    """
    if columns_to_remove is None:
        columns_to_remove = []
    if prefixes_to_remove is None:
        prefixes_to_remove = ()

    # Step 1: Drop the specific columns and prefix-columns
    to_drop = set(columns_to_remove)
    to_drop.update([col for col in df.columns if col.startswith(tuple(prefixes_to_remove))])
    df_cleaned = df.drop(columns=[col for col in to_drop if col in df.columns])
    print(f"[Step 1] Dropped {len(to_drop)} columns. Remaining: {df_cleaned.shape[1]}")

    # Step 2: Replace negative values, except -6 (refuses to answer) and -8 (don't know) with NaN
    value_map = []

    for col in df_cleaned.columns:
        if df_cleaned[col].dtype in ['int64', 'float64']:
            df_cleaned[col] = df_cleaned[col].apply(
                lambda x: x if (x >= 0 or x in [-6, -8] or pd.isna(x)) else np.nan
            )
            allowed_values = tuple(sorted(df_cleaned[col].dropna().unique()))
            value_map.append({"column": col, "value_set": allowed_values})

    print(f"[Step 2] Processed {len(value_map)} numeric columns.")

    value_pattern_df = pd.DataFrame(value_map)

    return value_pattern_df, df_cleaned

In [3]:
df = pd.read_csv('./data/mental-health-comorbidity-raw.csv')

admin_cols = ['RESPID', 'NCS1YR', 'AGE', 'STR', 'CASEID', 'COMPLETE', 'SECU', 'CASEWGT']
checkpoint_cols = ['M5A', 'IR3', 'IR11_4', 'IR36', 'IR47', 'PD0A', 'PD2', 'PD5', 'PD14', 'PD20', 'PD23', 'AG2', 'AG7', 'AG10', 
                   'FD4_1', 'FD6', 'FD7_1', 'FD9_1', 'PR1', 'PR11_1', 'PR15', 'FN1', 'FN4', 'FN24', 'CN1_2', 'CN4', 'CN4_1',
                   'CN7_1', 'CN7_2', 'CN8', 'CN14', 'DA36_2B', 'DA36_3A_1', 'DE20_3', 'DE20_6', 'CH23', 'CH38_1', 'CH74_1',
                   'CH104', 'AD0', 'AD2', 'AD7', 'AD29', 'AD31', 'AD36', 'AD43_2', 'OD2', 'OD27', 'CD3', 'CD17_1', 'CD24', 'SA1E_1',
                   'SA2', 'SA3', 'SA7A1', 'SA10', 'SA11E_1', 'SA12', 'SA18_5']

rem_cols = admin_cols + checkpoint_cols 

prefixes = ('IR48VALUES', 'PD27VALUES', 'PD28VALUES')

grouped_df, df_cleaned = preprocess_and_group_columns(
    df,
    columns_to_remove=rem_cols,
    prefixes_to_remove= prefixes
)

[Step 1] Dropped 109 columns. Remaining: 891
[Step 2] Processed 891 numeric columns.


In [4]:
# Number of columns with each unique value pattern
summary_df = (
    grouped_df
    .groupby("value_set")
    .size()
    .reset_index(name="num_columns")
    .sort_values("num_columns", ascending=False)
)

# Display results
print("Most common non-negative value patterns by column count:\n")
for _, row in summary_df.iterrows():
    print(f"{row['value_set']} → {row['num_columns']} columns")

Most common non-negative value patterns by column count:

(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(5.0)) → 174 columns
(np.float64(-6.0), np.float64(1.0), np.float64(5.0)) → 94 columns
(np.float64(1.0), np.float64(5.0)) → 86 columns
(np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)) → 53 columns
(np.float64(0.0), np.float64(1.0)) → 41 columns
(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)) → 40 columns
(np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)) → 26 columns
(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(5.0), np.float64(7.0)) → 16 columns
(np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0)) → 15 columns
(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(2.0)) → 13 columns
(np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.floa

In [5]:
def remove_sparse_value_sets(df, grouped_df, min_columns=4):
    """
    This function takes in a dataframe and a dictionary of grouped columns by their value sets.
    It removes columns that belong to value sets with fewer than `min_columns` columns.
    """
    value_set_counts = grouped_df['value_set'].value_counts()
    allowed_sets = value_set_counts[value_set_counts >= min_columns].index

    # Get columns with only min columns from allowed value sets
    columns_to_keep = grouped_df[grouped_df['value_set'].isin(allowed_sets)]['column']
    return df[columns_to_keep]

In [6]:
# Filter grouped_df to only value sets with ≥ 4 columns
value_set_counts = grouped_df['value_set'].value_counts()
filtered_sets = value_set_counts[value_set_counts >= 4]

print("Remaining value sets (with 4 or more columns):\n")
for value_set, count in filtered_sets.items():
    print(f"{value_set} → {count} columns")

Remaining value sets (with 4 or more columns):

(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(5.0)) → 174 columns
(np.float64(-6.0), np.float64(1.0), np.float64(5.0)) → 94 columns
(np.float64(1.0), np.float64(5.0)) → 86 columns
(np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)) → 53 columns
(np.float64(0.0), np.float64(1.0)) → 41 columns
(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)) → 40 columns
(np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)) → 26 columns
(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(5.0), np.float64(7.0)) → 16 columns
(np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0)) → 15 columns
(np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(2.0)) → 13 columns
(np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), 

In [7]:
# First filter grouped_df to only include value sets with ≥ 4 columns
value_set_counts = grouped_df['value_set'].value_counts()
filtered_sets = value_set_counts[value_set_counts >= 4].index
filtered_grouped_df = grouped_df[grouped_df['value_set'].isin(filtered_sets)]

# Now group and display
for value_set, group in filtered_grouped_df.groupby("value_set"):
    print(f"\nColumns with values {value_set} ({len(group)} columns):")
    for col in group['column']:
        print(f"  - {col}")


Columns with values (np.float64(-8.0), np.float64(-6.0), np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0)) (5 columns):
  - CN9
  - DA37
  - DA37A
  - DA38
  - DA38A

Columns with values (np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(2.0)) (13 columns):
  - FN3
  - FN8
  - SEX1
  - SEX2
  - SEX3
  - SEX4
  - SEX5
  - SEX6
  - SEX11
  - SEX12
  - SEX13
  - SEX14
  - SEX15

Columns with values (np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0)) (5 columns):
  - PR3
  - FN12
  - FN14
  - CN6A
  - CH1

Columns with values (np.float64(-8.0), np.float64(-6.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)) (40 columns):
  - PEA3
  - PEA27
  - PEA30
  - PEA42
  - NSD1A
  - NSD1C
  - NSD1D
  - NSD1L
  - PR6
  - FN14A
  - FN19
  - CN1_3
  - CN7_5A
  - CN10
  - CN11
  - DA41C
  - CH37A

In [8]:
filtered_df = remove_sparse_value_sets(df_cleaned, grouped_df, min_columns=4)
filtered_df.to_csv('./data/filtered_data.csv', index=False)

In [9]:
from data_columns import diagnosis

#Identifying diagnosis columns
diagnosis_set = set(diagnosis())
diagnosis_grouped_df = grouped_df[grouped_df['column'].isin(diagnosis_set)]


print("\nDiagnosis columns with value set [1, 5]:")
one_five_cols = diagnosis_grouped_df[
    diagnosis_grouped_df['value_set'] == (1, 5)
]['column'].tolist()
for col in one_five_cols:
    print(f"  - {col}")


print("\nDiagnosis columns with value set [0, 1, 2] or similar:")
zero_one_two_cols = []
for _, row in diagnosis_grouped_df.iterrows():
    val_set = set(row['value_set'])
    if val_set == {0, 1, 2} or ({1, 2}.issubset(val_set) and 0 in val_set):
        zero_one_two_cols.append(row['column'])
        print(f"  - {row['column']}")


Diagnosis columns with value set [1, 5]:
  - DSM_ADD
  - DSM_AGO
  - DSM_AGOWO
  - DSM_ALA
  - DSM_ALAH
  - DSM_ALD
  - DSM_ASA
  - DSM_BIPO1
  - DSM_BIPO2
  - DSM_CON
  - DSM_DRA
  - DSM_DRAH
  - DSM_DRD
  - DSM_DYS
  - DSM_DYSH
  - DSM_GAD
  - DSM_GADH
  - DSM_HYP
  - DSM_IED
  - DSM_IEDH
  - DSM_IMJ
  - DSM_IMN
  - DSM_MAN
  - DSM_MJD
  - DSM_MJDH
  - DSM_MND
  - DSM_MNDH
  - DSM_ODD
  - DSM_ODDH
  - DSM_PAT
  - DSM_PD
  - DSM_PD_AGO
  - DSM_PD_WOAGO
  - DSM_PMS
  - DSM_PTSD
  - DSM_SAD
  - DSM_SO
  - DSM_SP
  - DSM_TBD

Diagnosis columns with value set [0, 1, 2] or similar:
  - DSM_ASP
  - DSM_BOR
  - DSM_PEA
  - DSM_PEC


In [10]:
diagnosis_df = filtered_df

# observe positive diagnosis counts
one_five_counts = {
    col: (diagnosis_df[col] == 1).sum()
    for col in one_five_cols if col in diagnosis_df.columns
}


zero_one_two_counts = {
    col: ((diagnosis_df[col] == 1) | (diagnosis_df[col] == 2)).sum()
    for col in zero_one_two_cols if col in diagnosis_df.columns
}

# Get top 10 from each
top_1_5 = sorted(one_five_counts.items(), key=lambda x: x[1], reverse=True)[:10]
top_0_1_2 = sorted(zero_one_two_counts.items(), key=lambda x: x[1], reverse=True)[:10]

# Combine
combined_top = []
for col, count in top_1_5:
    combined_top.append((col, "1/5", count))
for col, count in top_0_1_2:
    combined_top.append((col, "0/1/2", count))
combined_top_sorted = sorted(combined_top, key=lambda x: x[2], reverse=True)


print("\nTop 10 Diagnosis Fields (Sorted by Count):")
for col, coding, count in combined_top_sorted:
    print(f"{col:<30} | Type: {coding:<6} | Count: {count}")


Top 10 Diagnosis Fields (Sorted by Count):
DSM_ASP                        | Type: 0/1/2  | Count: 4085
DSM_PEA                        | Type: 0/1/2  | Count: 3309
DSM_BOR                        | Type: 0/1/2  | Count: 3041
DSM_PEC                        | Type: 0/1/2  | Count: 2724
DSM_TBD                        | Type: 1/5    | Count: 1726
DSM_PAT                        | Type: 1/5    | Count: 1401
DSM_ALA                        | Type: 1/5    | Count: 1212
DSM_MJD                        | Type: 1/5    | Count: 995
DSM_ALAH                       | Type: 1/5    | Count: 960
DSM_SO                         | Type: 1/5    | Count: 808
DSM_MJDH                       | Type: 1/5    | Count: 804
DSM_SP                         | Type: 1/5    | Count: 762
DSM_IED                        | Type: 1/5    | Count: 500
DSM_GAD                        | Type: 1/5    | Count: 472


In [11]:
# now lets create a function where we can select a diagnoses, filter for usable columns
# and then filter out the other diagnosis columns
def get_filtered_features_for_diagnosis(df, diagnosis_col, positive_values, min_presence=0.3, columns_to_remove=None):
    if isinstance(positive_values, int):
        positive_values = [positive_values]
    if columns_to_remove is None:
        columns_to_remove = []

    diagnosed_df = df[df[diagnosis_col].isin(positive_values)].copy()
    print(f"[Subset] {diagnosed_df.shape[0]} rows match {diagnosis_col} ∈ {positive_values}")

    diagnosed_df.drop(columns=[col for col in columns_to_remove if col in diagnosed_df.columns], inplace=True)

    numeric_df = diagnosed_df.select_dtypes(include=['int64', 'float64'])

    valid_cols = [
        col for col in numeric_df.columns
        if (numeric_df[col] >= 0).mean() >= min_presence
    ]

    final_df = numeric_df[valid_cols]
    print(f"[Filter] {len(valid_cols)} features retained with ≥ {min_presence*100:.0f}% non-negative values")
    return final_df, valid_cols

In [12]:
cols_to_remove = diagnosis()
target_diagnosis = 'DSM_MJD'

filtered_features, retained_cols = get_filtered_features_for_diagnosis(
    df=filtered_df,
    diagnosis_col=target_diagnosis,
    positive_values=1,
    min_presence=0.3,
    columns_to_remove=cols_to_remove
)

[Subset] 995 rows match DSM_MJD ∈ [1]
[Filter] 301 features retained with ≥ 30% non-negative values


In [13]:
filtered_features.to_csv(f'./data/features_for_{target_diagnosis}.csv', index=False)

In [15]:
#Now Similar to what we did earlier, let's apply a more lenient filter to the raw mental health data
def pp(df, columns_to_remove=None, prefixes_to_remove=None):
    if columns_to_remove is None:
        columns_to_remove = []
    if prefixes_to_remove is None:
        prefixes_to_remove = ()

    # Step 1: Build full list of columns to remove
    to_drop = set(columns_to_remove)
    to_drop.update([col for col in df.columns if col.startswith(tuple(prefixes_to_remove))])

    # Step 2: Drop columns
    df_cleaned = df.drop(columns=[col for col in to_drop if col in df.columns])
    print(f"[Step 1] Dropped {len(to_drop)} columns. Remaining: {df_cleaned.shape[1]}")

    # Step 3: Group by value sets allowing only non-negative values + (-6, -8)
    grouped_columns = defaultdict(list)

    for col in df_cleaned.columns:
        if df_cleaned[col].dtype in ['int64', 'float64']:
            allowed_values = df_cleaned[col][(df_cleaned[col] >= 0) | (df_cleaned[col].isin([-6, -8]))].dropna().unique()
            value_set = tuple(sorted(allowed_values))
            grouped_columns[value_set].append(col)

    print(f"[Step 2] Grouped {len(df_cleaned.columns)} columns into {len(grouped_columns)} value patterns.")
    return grouped_columns

def rvs(df, grouped, min_columns=6):
    columns_to_keep = [
        col
        for value_set, cols in grouped.items()
        if len(cols) >= min_columns
        for col in cols
    ]
    
    return df[columns_to_keep]


df_neg = pd.read_csv('./data/mental-health-comorbidity-raw.csv')

grouped = pp(
    df_neg,
    columns_to_remove=rem_cols,
    prefixes_to_remove= prefixes
)
neg_filtered_df = rvs(df_neg, grouped, min_columns=6)


#below we will also save a raw data file which contains negative values as well for unsuperivsed learning, it also has a higher value set threshold
#we should expect to see less columns in this file than in filtered_data because we have a stricter value set threshold
neg_filtered_df.to_csv('./data/filtered_mental_health_data.csv', index=False)

# Now we will scale the data and impute missing values using KNNImputer
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(neg_filtered_df)

imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(scaled_data)

imputed_df = pd.DataFrame(imputed_data, columns=neg_filtered_df.columns, index=neg_filtered_df.index)
imputed_df.to_csv('./data/imputed_mental_health_data.csv', index=False)

[Step 1] Dropped 109 columns. Remaining: 891
[Step 2] Grouped 891 columns into 251 value patterns.


In [16]:
# here we can see how many columns
# are in each of our csv files thru the processing steps
csv_files = [
    './data/mental-health-comorbidity-raw.csv',
    './data/filtered_data.csv',
    './data/filtered_mental_health_data.csv',
    './data/imputed_mental_health_data.csv',
    './pp_outputs/features_for_DSM_MJD.csv'
]
results = []
for file_path in csv_files:
    try:
        df=pd.read_csv(file_path)
        results.append({"file": file_path, "rows": df.shape[0], "columns": df.shape[1]})
    except Exception as e:
        results.append({"file": file_path, "error": str(e)})
result_df = pd.DataFrame(results)
display(result_df)

,file,rows,columns
0,./data/mental-health-comorbidity-raw.csv,5001,1000
1,./data/filtered_data.csv,5001,648
2,./data/filtered_mental_health_data.csv,5001,594
3,./data/imputed_mental_health_data.csv,5001,594
4,./pp_outputs/features_for_DSM_MJD.csv,995,301
